In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px

In [ ]:
k_batches = 20

df_moves = pd.concat((
    pd.read_parquet(f"parsed/batch_{i}_moves.parquet")
    for i in range(1, k_batches + 1)
))

df_games = pd.concat((
    pd.read_parquet(f"parsed/batch_{i}_games.parquet")
    for i in range(1, k_batches + 1)
))

print(len(df_games))

In [ ]:
df_games["Elo"] = (df_games["WhiteElo"] + df_games["BlackElo"]) // 2

# df_moves["IsInaccuracy"] = df_moves["Move"].str.endswith("?!")
# df_moves["IsBlunder"] = df_moves["Move"].str.endswith("??")
# df_moves["IsMistake"] = df_moves["Move"].str.endswith("?") & (~df_moves["Move"].str.endswith("??"))
# df_moves["IsWrongMove"] = df_moves["IsInaccuracy"] | df_moves["IsBlunder"] | df_moves["IsMistake"]
# df_moves["IsBadMove"] = df_moves["IsBlunder"] | df_moves["IsMistake"]
# df_moves["IsOkayMove"] = ~(df_moves["IsInaccuracy"] | df_moves["IsBlunder"] | df_moves["IsMistake"])

# df_moves["EvalCentipawn"] = pd.to_numeric(df_moves["Eval"], errors="coerce").multiply(100).round()
# df_moves["EvalCentipawn"] = df_moves["EvalCentipawn"].clip(-1200, 1200)

# df_moves["AbsEval"] = df_moves["EvalCentipawn"].abs()

# df_moves["CentipawnLoss"] = df_moves["EvalCentipawn"].diff()
# df_moves.loc[(df_moves["MoveNumber"] == 1), "CentipawnLoss"] = 0
# df_moves.loc[ df_moves["MoveSide"] == 0, "CentipawnLoss" ] = -1 * df_moves.loc[ df_moves["MoveSide"] == 0, "CentipawnLoss" ].clip(upper=0)
# df_moves.loc[ df_moves["MoveSide"] == 1, "CentipawnLoss" ] = df_moves.loc[ df_moves["MoveSide"] == 1, "CentipawnLoss" ].clip(lower=0)


df_moves["Piece"] = df_moves["Move"].str[0]
# df_moves["IsCapture"] = df_moves["Move"].str.contains("x", regex=False)

# df_moves["IsCheck"] = df_moves["Move"].str.contains("+", regex=False)
# df_moves["IsCapture"] = df_moves["Move"].str.contains("x", regex=False)

In [ ]:
def sigmoid(x, a1, a2, a3, a4):
    s = 1 + np.exp( -a3*(x-a2) )
    y = a1 / s + a4
    return y

def inverse(x, a, b, c):
    return a/(x-b) + c

popt1 = [1.741759, 5.016247, 0.003036, -0.866097]
popt2 = [0.54774, -2.797004, 0.790583]
popt3 = [0.548174, 2.829586, -0.785972]

def eval_to_winodds(x):
    try:
        return sigmoid(float(x), *popt1)
    except:
        if float(x[1:]) > 0:
            return inverse(float(x[1:]), *popt2)
        elif float(x[1:]) < 0:
            return inverse(float(x[1:]), *popt3)
    return 0

In [ ]:
df_moves["Winodds"] = df_moves["Eval"].apply(eval_to_winodds)

df_moves["AdvLoss"] = df_moves["Winodds"].diff()
df_moves.loc[(df_moves["MoveNumber"] == 1), "AdvLoss"] = 0
df_moves.loc[ df_moves["MoveSide"] == 0, "AdvLoss" ] = -1 * df_moves.loc[ df_moves["MoveSide"] == 0, "AdvLoss" ].clip(upper=0)
df_moves.loc[ df_moves["MoveSide"] == 1, "AdvLoss" ] = df_moves.loc[ df_moves["MoveSide"] == 1, "AdvLoss" ].clip(lower=0)

In [ ]:
grid = []
search_space = list(range(3, 50+1, 1))
for i in search_space:
    print(f"{i=}", end="\r")
    
    # df_moves["Feature"] = df_moves["MoveNumber"].where(
    #     (df_moves["Piece"] == "Q")
    # )
    # agg = df_moves.groupby("GameId", as_index=False).agg(FeatureA=("Feature", "min"))
    # df_merged = df_games.merge(agg, on="GameId")
    # df_merged["FeatureA"] = df_merged["FeatureA"].fillna(25) <= i

    # df_moves["Feature"] = df_moves["MoveNumber"].where(
    #     (df_moves["Move"].str.startswith("O-"))
    # )
    # agg = df_moves.groupby("GameId", as_index=False).agg(FeatureA=("Feature", "min"))
    # df_merged = df_games.merge(agg, on="GameId")
    # df_merged["FeatureA"] = df_merged["FeatureA"].fillna(25) <= i

    df_moves["Feature"] = (df_moves["Piece"] == "N") & (df_moves["MoveNumber"] <= i)
    agg = df_moves.groupby("GameId", as_index=False).agg(FeatureA=("Feature", "mean"))
    df_merged = df_games.merge(agg, on="GameId")
    df_merged["FeatureA"] = df_merged["FeatureA"].fillna(0)

    # df_moves["Feature"] = (df_moves["Piece"].str.islower()) & (df_moves["MoveNumber"] <= i)
    # agg = df_moves.groupby("GameId", as_index=False).agg(FeatureA=("Feature", "mean"))
    # df_merged = df_games.merge(agg, on="GameId")
    # df_merged["FeatureA"] = df_merged["FeatureA"].fillna(0)

    # df_moves["Feature"] = (df_moves["Move"].str.contains("x")) & (df_moves["MoveNumber"] <= i)
    # agg = df_moves.groupby("GameId", as_index=False).agg(FeatureA=("Feature", "mean"))
    # df_merged = df_games.merge(agg, on="GameId")
    # df_merged["FeatureA"] = df_merged["FeatureA"].fillna(0)

    # df_moves["Feature"] = (df_moves["Move"].str.contains("+", regex=False)) & (df_moves["MoveNumber"] <= i)
    # agg = df_moves.groupby("GameId", as_index=False).agg(FeatureA=("Feature", "mean"))
    # df_merged = df_games.merge(agg, on="GameId")
    # df_merged["FeatureA"] = df_merged["FeatureA"].fillna(0)

    
    
    corr = df_merged["Elo"].corr(
        df_merged["FeatureA"],
        method="spearman"
    )
    grid.append(corr)
grid = np.array(grid)

In [ ]:
fig = px.line(
    x=search_space,
    # y=np.abs(grid),
    y=grid,
    template="plotly_white"
)
fig.data[0].mode="lines+markers"
fig.data[0].line.color="#008C45"
# fig.data[0].line.width=4*2
# fig.data[0].marker.size=10*2

fig.update_xaxes(title="")
fig.update_yaxes(title="")
# fig.update_layout(height=1080, width=1920, font_size=40, font_family="Consolas")

fig.show()
# fig.write_image("presentation/images/gridsearch.png", scale=2)

In [ ]:
df_moves["Feature"] = (df_moves["Piece"] == "N") & (df_moves["MoveNumber"] <= 10)
agg = df_moves.groupby("GameId", as_index=False).agg(FeatureA=("Feature", "mean"))
df_merged = df_games.merge(agg, on="GameId")
df_merged["FeatureA"] = df_merged["FeatureA"].fillna(0)

In [ ]:
px.histogram(df_merged["FeatureA"])